In [24]:
import numpy as np
import scipy.stats
import pandas as pd
import matplotlib.pyplot as plt

In [25]:
AAPL_df=pd.read_csv('AAPL.csv',index_col=0)
TSLA_df=pd.read_csv('TSLA.csv',index_col=0)
TXN_df=pd.read_csv('TXN.csv',index_col=0)
GME_df=pd.read_csv('GME.csv',index_col=0)
GM_df=pd.read_csv('GM.csv',index_col=0)

In [26]:
AAPL_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,38.722500,39.712502,38.557499,39.480000,38.277519,148158800
2019-01-03,35.994999,36.430000,35.500000,35.547501,34.464802,365248800
2019-01-04,36.132500,37.137501,35.950001,37.064999,35.936077,234428400
2019-01-07,37.174999,37.207500,36.474998,36.982498,35.856094,219111200
2019-01-08,37.389999,37.955002,37.130001,37.687500,36.539623,164101200


In [50]:
AAPL=AAPL_df['Adj Close'].pct_change().dropna()
TSLA=TSLA_df['Adj Close'].pct_change().dropna()
GME=GME_df['Adj Close'].pct_change().dropna()
GM=GM_df['Adj Close'].pct_change().dropna()
TXN=TXN_df['Adj Close'].pct_change().dropna()
#AAPL= AAPL_df.loc[:,'Adj Close']
#TSLA= TSLA_df.loc[:,'Adj Close']
#GME= GME_df.loc[:,'Adj Close']
#GM= GM_df.loc[:,'Adj Close']
#TXN= TXN_df.loc[:,'Adj Close']

stock_list = [AAPL_df, TSLA_df, GME_df, GM_df, TXN_df]
'''for stock in stock_list:
    stock=stock['Adj Close'].pct_change()'''
AAPL

Date
2019-01-03   -0.099607
2019-01-04    0.042689
2019-01-07   -0.002226
2019-01-08    0.019063
2019-01-09    0.016982
                ...   
2022-03-15    0.029677
2022-03-16    0.029015
2022-03-17    0.006454
2022-03-18    0.020919
2022-03-21    0.008538
Name: Adj Close, Length: 810, dtype: float64

In [51]:

stock_list = [AAPL, TSLA, GME, GM, TXN]
df_stock = pd.concat(stock_list,axis=1)
df_stock.columns = ['AAPL', 'TSLA', 'GME', 'GM', 'TXN']
df_stock = df_stock.sort_index(ascending=True)
df_stock.head()

,AAPL,TSLA,GME,GM,TXN
Date,,,,,
2019-01-03,-0.099607,-0.031472,-0.007651,-0.041320,-0.058973
2019-01-04,0.042689,0.057697,0.175019,0.033488,0.045229
2019-01-07,-0.002226,0.054361,0.015748,0.030903,0.017223
2019-01-08,0.019063,0.001164,0.021318,0.013097,-0.001270
2019-01-09,0.016982,0.009483,0.010753,0.010629,0.019708


In [78]:
#quantify the fama french 5 factor
df_ff=pd.read_csv('FF_Research_Data_5_Factors.csv',index_col=0)
df_ff.head()
#since the fama french has only integer datetime, so we need to align their date format
#try millions time..
df_ff.index=pd.to_datetime(df_ff.index, format = '%Y%m%d').strftime('%Y-%m-%d')
df_ff.head()


,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,
1963-07-01,-0.67,0.01,-0.35,0.03,0.11,0.012
1963-07-02,0.79,-0.31,0.24,-0.08,-0.25,0.012
1963-07-03,0.63,-0.16,-0.09,0.13,-0.24,0.012
1963-07-05,0.40,0.09,-0.26,0.07,-0.28,0.012
1963-07-08,-0.63,0.07,-0.19,-0.27,0.06,0.012


In [91]:
#merge two return to do the linear regression
df = pd.merge(df_stock, df_ff, left_index=True, right_index=True, how='inner')
df = df.fillna(0)
#rf use avg of 2019-2022 avg(2.14%,0.04%0.44%)
rf = ((1.0214+1.00004+1.0044)/3) ** (1/360) - 1
df = df - rf
df2 = df.copy()
df = df['2019-01-02':][:250]
#df.shape 250 rolling window
#df.head()

(250, 11)

In [83]:
import statsmodels.api as sm

stock_names = {
    'AAPL': 'Aaple',
    'TSLA': 'Tesla',
    'GME': 'GME',
    'GM': 'GM',
    'TXN': 'Texas Instrument'
}
#fit the multi linear regression
'''for stock in ['AAPL', 'TSLA', 'GME', 'GM', 'TXN']:
    model = sm.OLS(df[stock], sm.add_constant(
        df[['Mkt-RF','SMB', 'HML','RMW','CMA']].values))
    result = model.fit()
    print(stock_names[stock] + '\n')
    print(result.summary())
    #CHECK BETA EXPLANATORY POWER AND R-SQUARE
    print('\n\n')
    print('\n\n')'''
#AAPL'S RESULT
X=df[['Mkt-RF','SMB', 'HML','RMW','CMA']]
Y=df['AAPL']
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)


                                 OLS Regression Results                                
Dep. Variable:                   AAPL   R-squared (uncentered):                   0.748
Model:                            OLS   Adj. R-squared (uncentered):              0.746
Method:                 Least Squares   F-statistic:                              457.8
Date:                Thu, 24 Mar 2022   Prob (F-statistic):                   6.13e-228
Time:                        15:57:19   Log-Likelihood:                          2410.0
No. Observations:                 776   AIC:                                     -4810.
Df Residuals:                     771   BIC:                                     -4787.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [84]:
#TSLA'S RESULT
X=df[['Mkt-RF','SMB', 'HML','RMW','CMA']]
Y=df['TSLA']
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

                                 OLS Regression Results                                
Dep. Variable:                   TSLA   R-squared (uncentered):                   0.313
Model:                            OLS   Adj. R-squared (uncentered):              0.308
Method:                 Least Squares   F-statistic:                              70.11
Date:                Thu, 24 Mar 2022   Prob (F-statistic):                    1.83e-60
Time:                        15:58:19   Log-Likelihood:                          1489.2
No. Observations:                 776   AIC:                                     -2968.
Df Residuals:                     771   BIC:                                     -2945.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [85]:
#GME'S RESULT
X=df[['Mkt-RF','SMB', 'HML','RMW','CMA']]
Y=df['GME']
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

                                 OLS Regression Results                                
Dep. Variable:                    GME   R-squared (uncentered):                   0.145
Model:                            OLS   Adj. R-squared (uncentered):              0.139
Method:                 Least Squares   F-statistic:                              26.15
Date:                Thu, 24 Mar 2022   Prob (F-statistic):                    1.91e-24
Time:                        15:58:35   Log-Likelihood:                          675.33
No. Observations:                 776   AIC:                                     -1341.
Df Residuals:                     771   BIC:                                     -1317.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [86]:
#GM'S RESULT
X=df[['Mkt-RF','SMB', 'HML','RMW','CMA']]
Y=df['GM']
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

                                 OLS Regression Results                                
Dep. Variable:                     GM   R-squared (uncentered):                   0.579
Model:                            OLS   Adj. R-squared (uncentered):              0.576
Method:                 Least Squares   F-statistic:                              212.0
Date:                Thu, 24 Mar 2022   Prob (F-statistic):                   3.98e-142
Time:                        15:58:45   Log-Likelihood:                          2014.4
No. Observations:                 776   AIC:                                     -4019.
Df Residuals:                     771   BIC:                                     -3995.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [87]:
#TXN'S RESULT
X=df[['Mkt-RF','SMB', 'HML','RMW','CMA']]
Y=df['TXN']
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) 

print_model = model.summary()
print(print_model)

                                 OLS Regression Results                                
Dep. Variable:                    TXN   R-squared (uncentered):                   0.645
Model:                            OLS   Adj. R-squared (uncentered):              0.643
Method:                 Least Squares   F-statistic:                              280.1
Date:                Thu, 24 Mar 2022   Prob (F-statistic):                   1.25e-170
Time:                        15:59:00   Log-Likelihood:                          2305.2
No. Observations:                 776   AIC:                                     -4600.
Df Residuals:                     771   BIC:                                     -4577.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------